In [65]:
from ExpAlgae.model.COBRAmodel import *
from ExpAlgae import DATA_PATH

In [66]:
model = MyModel(join(DATA_PATH, 'models/model_plutheri.xml'), 'e_Biomass__in')

Loading



2686093 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id


Reactions: 2214
Metabolites: 2062
Genes: 1177
Model loaded


In [67]:
model.create_sink("C00025__in")
model.create_sink("C00169__in")
model.create_sink("C00002__in")
model.create_sink("C00143__in")
model.create_demand("C00390__in")
model.create_demand("C00415__in")
model.create_demand("C00147__in")

Reaction identifier,Dm_C00147__in
Name,
Memory address,0x01b6c4ec0970
Stoichiometry,C00147__in --> Adenine -->
GPR,
Lower bound,0
Upper bound,10000


In [68]:
model.create_tRNAs_reactions()

In [69]:
model.test_reaction("R00190__in")

Ignoring reaction 'Dm_C00147__in' since it already exists.


,Flux
Adenine,0.0
5-Phospho-alpha-D-ribose 1-diphosphate,1000.0


In [70]:
model.test_reaction("R02135__in")

,Flux
H2O,8822.153575
Thiamin monophosphate,333.333333


In [71]:
model.test_reaction("e_Cofactor__in")

,Flux
Ubiquinone,0.000000
Thiamine,333.333333
NAD+,500.000000
S-Adenosyl-L-methionine,1000.000000
FAD,500.000000
Tetrahydrofolate,2272.727273
Glutathione,1000.000000
CoA,784.722222
Heme,125.000000
Riboflavin,500.000000
